# Quantum Phase Estimation

Quantum phase estimation algorithm is used to estimate the phase $\phi$ in $U\left| v \right\rangle = e^{2\pi i \phi}\left| v \right\rangle$, where $U$ is a unitary operator and $\left| v \right\rangle$ is it's eigenvector <a name="ref-1"/>[(Nielsen and Chuang, 2010)](#cite-MikeIke). 

Below is an example to estimate the phase of the unitary operator $U = Z^{1/16}$. Note that $U$ is given by
\begin{equation}U = \begin{bmatrix}
1 & 0\\
0 & e^{i\pi/16}
\end{bmatrix},
\end{equation}

and the exact solution in this case is $\phi = 1/32$ for the eigenvector $\left| v \right\rangle = \left| 1 \right\rangle$. Implementation using __[Cirq](https://quantumai.google/cirq)__.

In [2]:
#Quantum phase estimation

In [3]:
import random
import numpy as np
import cirq
import statistics
from statistics import mode
from cirq import Z, H, X, CNOT, S ,measure

In [4]:
#QFT+ gate
#QFT+ applied to quantum circuit
def QFTD(QC,QBTS,N):

    #Apply S gates with negative exponents
    for i in range(N):
        QC.append(H(QBTS[i])) #Add H gate
        M = S**(-1.0)
        j = i + 1
        while j < N:
            QC.append(M(QBTS[i]).controlled_by(QBTS[j]))
            M = M**0.5
            j += 1
            
            
    #Swap the qubits
    nsw = int(N/2)
    for i in range(nsw):
        QC.append(cirq.SWAP(QBTS[i],QBTS[N-1-i]))

In [5]:
n = 6 #Counting qubits
qbs = cirq.LineQubit.range(n+1) #One additional qubit for state |1>
qc = cirq.Circuit()

#Prepate the initial state
#Apply H gate to each of the first n qubits
for i in range(n):
    qc.append(H(qbs[i]))

#Add X gate to last qubit 
qc.append(X(qbs[n]))

print("Circuit:")
print(qc)

Circuit:
0: ───H───

1: ───H───

2: ───H───

3: ───H───

4: ───H───

5: ───H───

6: ───X───


In [6]:
#Apply controlled U gates on last qubit
#Least significant qubit of first n qubits controls U
#Most significant qubit of first n qubits controls U**(2**(n-1))

U = Z**(0.0625) #The U gate
U = U**(2**(n-1))
for i in range(n):
    qc.append(U(qbs[n]).controlled_by(qbs[i]))
    U = U**0.5
    

print("Circuit:")
print(qc)

Circuit:
0: ───H───@──────────────────────────────────────
          │
1: ───H───┼─────@────────────────────────────────
          │     │
2: ───H───┼─────┼───@────────────────────────────
          │     │   │
3: ───H───┼─────┼───┼───@────────────────────────
          │     │   │   │
4: ───H───┼─────┼───┼───┼───@────────────────────
          │     │   │   │   │
5: ───H───┼─────┼───┼───┼───┼─────────@──────────
          │     │   │   │   │         │
6: ───X───Z^0───Z───S───T───Z^(1/8)───Z^(1/16)───


In [7]:
#Apply QFT+ to the first n qubits
QFTD(qc,qbs,n)

#print("Circuit:")
#print(qc)

In [8]:
#Add measuremnt operator to each of the first n qubits
qc.append(measure(qbs[0],qbs[1],qbs[2],qbs[3],qbs[4],qbs[5],key='Results'))


In [9]:
#Simulate the circuit
Order_List = [qbs[i] for i in range(n+1)] #Order list for the results
simulator = cirq.Simulator()
results = simulator.simulate(qc, qubit_order=[qbs[i] for i in range(n+1)])
print(results)

samples=simulator.run(qc,repetitions=50000)
print(samples.histogram(key='Results'))

measurements: Results=000010
output vector: |0000101⟩
Counter({2: 50000})







Final state should be $\left| 2^{n}\phi \right\rangle$, where $n$ is the number of counting qubits.


In [10]:
Final_state = 2 #Result from simulation
#(2**n)*phi = state
phi = Final_state/(2**n)
print('Phi = %0.5f\n'%(phi)) #phi = 1/32

Phi = 0.03125



<!--bibtex

@book{MikeIke,
    title = {Quantum Computation and Quantum Information},
    author = {Nielsen, Michael A. and Chuang, Issac L},
    year = {2010},
    publisher = {Cambridge University Press}
}

-->

# References

<a name="cite-MikeIke"/><sup>[^](#ref-1) </sup>Nielsen, Michael A. and Chuang, Issac L. 2010. _Quantum Computation and Quantum Information_.

